In [164]:
import pandas as pd

In [165]:
df=pd.read_csv('fake_news.csv',on_bad_lines='skip',engine='python')

In [166]:
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20817,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20818,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20819,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20820,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [167]:
df.isnull().sum()

id           0
title      565
author    1975
text        59
label       22
dtype: int64

In [168]:
df.dropna(axis=0,inplace=True)

In [169]:
df['label'].unique()

array(['1', '0', ' как люди воспринимают своё положение.',
       ' чтобы это была дорога с двусторонним движением.'], dtype=object)

In [170]:
df=df[df['label'].isin(['0','1'])]
df=df.reset_index(drop=True)

In [171]:
x=df.drop(labels='label',axis=1)

In [172]:
y=df['label']

In [173]:
x.shape,y.shape

((18283, 4), (18283,))

In [174]:
x

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
18278,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18279,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18280,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18281,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [175]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Embedding, Input, LSTM,Dense, Dropout, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [176]:
vocabulary_size=10000

In [177]:
messages=x.copy()

In [178]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
18278,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18279,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18280,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18281,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [179]:
import nltk
import re
from nltk.corpus import stopwords

In [180]:
from nltk.stem import WordNetLemmatizer

In [181]:
print(type(messages['title']))

<class 'pandas.core.series.Series'>


In [182]:
corpus=[]
stop_words=stopwords.words('english')
for i in range(0,len(messages['title'])):
    review=re.sub('[^a-zA-Z]', ' ',str(messages['title'][i]))
    review=review.lower()
    review=review.split()
    
    review=[WordNetLemmatizer().lemmatize(word) for word in review if word not in stop_words]
    review=' '.join(review)
    corpus.append(review)

In [183]:
corpus

['house dem aide even see comey letter jason chaffetz tweeted',
 'flynn hillary clinton big woman campus breitbart',
 'truth might get fired',
 'civilian killed single u airstrike identified',
 'iranian woman jailed fictional unpublished story woman stoned death adultery',
 'jackie mason hollywood would love trump bombed north korea lack trans bathroom exclusive video breitbart',
 'beno hamon win french socialist party presidential nomination new york time',
 'back channel plan ukraine russia courtesy trump associate new york time',
 'obama organizing action partner soros linked indivisible disrupt trump agenda',
 'bbc comedy sketch real housewife isi cause outrage',
 'russian researcher discover secret nazi military base treasure hunter arctic photo',
 'u official see link trump russia',
 'yes paid government troll social medium blog forum website',
 'major league soccer argentine find home success new york time',
 'well fargo chief abruptly step new york time',
 'anonymous donor pay 

In [184]:
len(corpus)

18283

In [185]:
onehot_repr=[one_hot(word,vocabulary_size) for word in corpus]

In [186]:
print(onehot_repr)

[[1132, 8962, 4972, 8824, 3322, 9453, 9251, 6193, 5682, 9644], [292, 6476, 5325, 9412, 2059, 4214, 8339], [8361, 5777, 9019, 9393], [4473, 6072, 8915, 9246, 4956, 3364], [2404, 2059, 5055, 9849, 5515, 1982, 2059, 5852, 1999, 8932], [2848, 4820, 3772, 9760, 9366, 4713, 8154, 2678, 5565, 680, 5145, 8104, 3694, 5847, 8339], [2971, 5173, 5165, 3477, 4718, 3132, 7675, 897, 2893, 4557, 3470], [7310, 9168, 5630, 2810, 7046, 4231, 4713, 1744, 2893, 4557, 3470], [4505, 4030, 8453, 353, 4368, 9483, 8735, 1727, 4713, 848], [9904, 8640, 3994, 347, 5037, 1151, 7420, 7698], [235, 3556, 1423, 22, 633, 5432, 1534, 6499, 1149, 2888, 1039], [9246, 8966, 3322, 7775, 4713, 7046], [8268, 9515, 7028, 5051, 9963, 5258, 9947, 3304, 2052], [7863, 9981, 1498, 3126, 872, 5445, 5359, 2893, 4557, 3470], [520, 8008, 7827, 5837, 2396, 2893, 4557, 3470], [3183, 7744, 8907, 9679, 9763, 6231, 9372, 6558, 2643, 9086], [2556, 8527, 6476], [1374, 995, 2418, 754, 4713, 645, 695, 8339], [3258, 5663, 5325, 3804, 8804, 9384, 

In [187]:
max(len(i) for i in onehot_repr)

47

In [188]:
sentence_length=47

In [189]:
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sentence_length)

In [190]:
embedded_docs

array([[1132, 8962, 4972, ...,    0,    0,    0],
       [ 292, 6476, 5325, ...,    0,    0,    0],
       [8361, 5777, 9019, ...,    0,    0,    0],
       ...,
       [2634, 8278, 8397, ...,    0,    0,    0],
       [ 175, 7046,  974, ...,    0,    0,    0],
       [ 327, 8830, 7230, ...,    0,    0,    0]],
      shape=(18283, 47), dtype=int32)

In [191]:
each_vector_size=100

In [192]:
model=Sequential([Input(shape=(sentence_length,))])

In [193]:
model.add(Embedding(input_dim=vocabulary_size,output_dim=each_vector_size))
#this is basically, input_dim is like telling give me a matrix with 10k rows, and output_dim means each word should have n number of feature representations
#also you can interpret it as i have 10k words and each word has to be explained with 100 unique features

In [194]:
model.add(Dropout(0.2))

In [195]:
model.add(Bidirectional(LSTM(200))) #this basically means we'll have 200 neurons backward, and 200 neuron forward

In [196]:
model.add(Dropout(0.2))

In [197]:
model.add(Dense(1,activation='sigmoid'))

In [198]:
model.compile(loss='binary_crossentropy',optimizer='adamW',metrics=['accuracy'])

In [199]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)         │ (None, 47, 100)        │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 47, 100)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_7 (Bidirectional) │ (None, 400)            │       481,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 400)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │           401 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,482,001 (5.65 MB)

 Trainable params: 1,482,001 (5.65 MB)

 Non-trainable params: 0 (0.00 B)

In [200]:
import numpy as np

In [201]:
x=np.array(embedded_docs)

In [202]:
from sklearn.preprocessing import LabelEncoder

In [203]:
y=LabelEncoder().fit_transform(y)

In [204]:
y=np.array(y)

In [205]:
x.shape,y.shape

((18283, 47), (18283,))

In [206]:
from sklearn.model_selection import train_test_split

In [207]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3)

In [208]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=128)

Epoch 1/10


100/100 ━━━━━━━━━━━━━━━━━━━━ 28s 247ms/step - accuracy: 0.8169 - loss: 0.3571 - val_accuracy: 0.9099 - val_loss: 0.2061
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 22s 218ms/step - accuracy: 0.9425 - loss: 0.1421 - val_accuracy: 0.9245 - val_loss: 0.1882
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 22s 219ms/step - accuracy: 0.9710 - loss: 0.0867 - val_accuracy: 0.9240 - val_loss: 0.2090
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 21s 214ms/step - accuracy: 0.9823 - loss: 0.0571 - val_accuracy: 0.9209 - val_loss: 0.2606
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 22s 217ms/step - accuracy: 0.9887 - loss: 0.0381 - val_accuracy: 0.9165 - val_loss: 0.3025
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 22s 216ms/step - accuracy: 0.9941 - loss: 0.0218 - val_accuracy: 0.9187 - val_loss: 0.3488
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 22s 217ms/step - accuracy: 0.9965 - loss: 0.0140 - val_accuracy: 0.9139 - val_loss: 0.4226
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 21s 214ms/step - accuracy: 0.9970 - loss: 0.0110 - val

In [212]:
y_pred=model.predict(x_test)

172/172 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step


In [213]:
y_pred=np.where(y_pred>0.5,1,0)

In [214]:
y_pred

array([[0],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]], shape=(5485, 1))

In [215]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [216]:
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

0.9134001823154057
              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3043
           1       0.89      0.92      0.90      2442

    accuracy                           0.91      5485
   macro avg       0.91      0.91      0.91      5485
weighted avg       0.91      0.91      0.91      5485

[[2765  278]
 [ 197 2245]]
